In [89]:
import requests
import urllib2
from bs4 import BeautifulSoup
import csv

In [67]:
URL_TEAM = "https://www.basketball-reference.com/play-index/event_finder.cgi?request=1&player_id=&id_val=&event_code=ast&year_id=2017&is_playoffs=N&team_id="
URL_OFFSET = "&opp_id=&quarter=&time_remain_quarter=&margin=&score=&offset="

# usage, URL_TEAM + "TEAM_NAME" + URL_OFFSET + "INDEX"

In [68]:
def getSoupFromURL(url):

    try:
        r = requests.get(url)
    except:
        return None

    return BeautifulSoup(r.text, "html5lib")

def getUrlFromTeamOffset(team, offset):
    return URL_TEAM + team + URL_OFFSET + str(offset)

In [110]:
nba_teams = ["ATL", "NJN", "BOS", "CHA", "CHI", "CLE", "DAL", "DEN", "DET", "GSW", "HOU", "IND", "LAC", "LAL", 
             "MEM", "MIA", "MIL", "MIN", "NOH", "NYK", "OKC", "ORL", "PHI", "PHO", "POR", "SAC", "SAS", "TOR", 
             "UTA", "WAS"]

In [103]:
def get_all_plays_in_table(offset, team_name, data):
    url = getUrlFromTeamOffset(team_name, offset)
    soup = getSoupFromURL(url)
    table = soup.find('table', attrs={"class" : "sortable"})
    table_body = table.find('tbody')

    rows = table_body.find_all('tr')
    for row in rows:
        cols = row.find_all('td', attrs={"data-stat" : "play_desc"})
        for elem in cols:
            if elem.text.strip() == " ":
                continue
            string =  elem.text.strip()
            makes_split = string.split("makes")
            scorer = makes_split[0].strip()
            assist_by_split = string.split("assist by")
            passer = assist_by_split[1].strip()[:-1]
            shot_type = makes_split[1].split("shot")[0].strip()
            distance = 0
            if "at rim" not in string:
                distance = int(string.split("from")[1].split("ft")[0].strip())
            data.append([passer, scorer, shot_type, distance])
    return data

In [83]:
def get_number_of_urls(team_name):
    """
    Offset doesn't matter here
    """
    url = getUrlFromTeamOffset(team_name, 0)
    soup = getSoupFromURL(url)
    table = soup.find('table', attrs={"class" : "stats_table"})
    table_body = table.find('tbody')
    indexes = []
    rows = table_body.find_all('tr')
    data = []
    for row in rows:
        cols = row.find_all('td')
        for elem in cols:
            if elem.text.strip() == " ":
                continue
            text = elem.text.strip()
            indexes.append(int(text.split("-")[0])-1)
    return indexes
    

In [78]:
indexes = get_number_of_urls("MIL")

In [104]:
data = []
get_all_plays_in_table(0, "MIL", data)

[[u'M. Plumlee', u'M. Dellavedova', u'3-pt', 26],
 [u'M. Dellavedova', u'G. Antetokounmpo', u'2-pt', 0],
 [u'G. Monroe', u'J. Parker', u'2-pt', 0],
 [u'M. Brogdon', u'G. Antetokounmpo', u'2-pt', 0],
 [u'M. Brogdon', u'G. Antetokounmpo', u'2-pt', 0],
 [u'G. Monroe', u'G. Antetokounmpo', u'2-pt', 0],
 [u'G. Antetokounmpo', u'M. Brogdon', u'2-pt', 1],
 [u'M. Teletovic', u'J. Henson', u'2-pt', 0],
 [u'J. Terry', u'M. Beasley', u'2-pt', 20],
 [u'M. Brogdon', u'M. Teletovic', u'2-pt', 0],
 [u'M. Dellavedova', u'M. Teletovic', u'2-pt', 0],
 [u'M. Dellavedova', u'R. Vaughn', u'2-pt', 17],
 [u'G. Antetokounmpo', u'M. Dellavedova', u'2-pt', 23],
 [u'J. Parker', u'G. Monroe', u'2-pt', 1],
 [u'G. Monroe', u'G. Antetokounmpo', u'2-pt', 1],
 [u'J. Parker', u'G. Antetokounmpo', u'3-pt', 25],
 [u'G. Antetokounmpo', u'M. Teletovic', u'2-pt', 0],
 [u'J. Parker', u'G. Monroe', u'2-pt', 18],
 [u'M. Brogdon', u'J. Parker', u'3-pt', 27],
 [u'M. Beasley', u'M. Brogdon', u'2-pt', 0],
 [u'G. Antetokounmpo', u'

In [105]:
with open("../../Data/asists_inside_team/temp.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerow(['Passer', 'Scorer', 'ShotType', 'Distance'])
    writer.writerows(data)

In [112]:
nba_teams = ["PHO", "POR", "SAC", "SAS", "TOR", 
             "UTA", "WAS"]
for team in nba_teams:
    
    data = []
    indexes = get_number_of_urls(team)
    for index in indexes:
        get_all_plays_in_table(index, team, data)
    
    if team == "NJN":
        team = "BKN"
    elif team == "NOH":
        team = "NOP"
    file_name = "../../Data/asists_inside_team/" + team + ".csv"
    with open(file_name, "wb") as f:
        writer = csv.writer(f)
        writer.writerow(['Passer', 'Scorer', 'ShotType', 'Distance'])
        writer.writerows(data)
